# **Analyse qualité des données (DQM)**

1. Analyse conceptuelle et sémantique des data (object, relation et type -> type schéma UML)
2. Analyse technique brute des data
   1. Vérification de la consistance de la base (ex: même nombre de colonne)
   2. Vérification des contraites conceptuelles et techniques issus de la sématique
      1. Vérification homogénisation des types de données par X
      2. Vérification unicité des clés et existance (primaires, étrangère, NOT NULL)
      3. Vérification et identification des données manquantes et caractéristiques de cela
3. Vérification de la cohérence conceptuelle
   1. Lien d'antériorité des objects (au sein d'un même object id/date et entre les objects (recette/review))

## ***I) Analyse conceptuelle et sémantique***

### *data interaction (RAW_interactions.csv)*

- **user_id** : *Integer* (clée étrangère)

    L'utilisateur ayant effectué l'interaction (ici, un commentaire)

- **recipe_id** : *Integer* (clée étrangère)

    La recette de l'interaction

- **date** : *str en format ('aaaa-mm-dd')*

    La date de publication de l'intéraction

- **rating** : *int* (interval)

    La note fournie à la recette (le plus probable)

- **review** : *Text*

    Le commentaire associé à l'intéraction

=>clé primaire (user_id, recipe_id, date) [__a minima__]

C'est une table de relation entre USER et RECETTE.

L'intéraction représente temporellement un commentaire ou une note sur une recette donnée par un utilisateur identifié (?) du site.

### *data recipe (RAW_recipes.csv)*

- **name** : *str*

    Le nom de la recette

- **id** : *Integer* (clée primaire)

    Identifiant unique de la recette

- **minute** : *Integer*

    Temps de réalisation de la recette

- **contributor_id** : *Integer* (clée étrangère)

    L'utilisateur ayant créé la recette

- **submitted** : *str en format ('aaaa-mm-dd')*

    La date de publication de la recette

- **tag** : *list[str]* (voir si objet externe enregistré en dur)

    les tags associés à une recette

- **nutrition** : *list[5 x float]*

    Les informations de nutrition

- **n_step** : *Integer*

    Le nombre d'étape pour réaliser la recette

- **step** : *list[Text]*

    Les étapes de réalisation de la recette

- **description** : *Text*

    La descrption de la recette

- **ingredient** : *list[str]* (voir si objet externe enregistré en dur)

    Les ingrédents pour réaliser la recette

- **n_ingredient** *Integer* (voir si champ technique généré automatiquement ou via user)

    Le nombre d'ingrédient pour la recette

C'est la table RECETTE qui a une relation avec la table USER et aussi potentiellement une table TAG, INGREDIENT.

### Résumé UML :

![Figure 1 : schéma UML](Data_UML.png)

- interaction(**#user_id, #recipe_id, date,** rating, review)

| Nom       | Type            | Options                     | Description          |
|:----------|:----------------|:----------------------------|:---------------------|
| user_id   | Integer         | NOT NULL                    | id user commentant   |
| recipe_id | Integer         | NOT NULL                    | id recette commentée |
| date      | Date=aaaa-mm-dd | NOT NULL, Default=CURDATE() | Date du commentaire  |
| rating    | Integer         | NOT NULL, Default=0         | Note (0 à 5)         |
| review    | TEXT            | NOT NULL, Default=''        | Commentaire          |

**ATTENTION :** Cette table ne présente pas de contraite conforme. Il peut avoir des doublons, sauf si un id technique ne nous ait pas connu. (cas d'erreur = le même jour, user avec même note et commentaire)

- recipe(name, **id**, minutes, #contributor_id, submitted, tags, nutrition, n_steps, steps, description, ingredients, n_ingredients)

| Nom            | Type                 | Options                       | Description               |
|:---------------|:---------------------|:------------------------------|:--------------------------|
| name           | String               | NOT NULL, Default=''          | Nom recette               |
| id             | Integer              | Auto-increment, NOT NULL      | id recette                |
| minutes        | Integer              | NOT NULL, Default=0           | Temps de la recette       |
| contributor_id | Integer              | NOT NULL                      | id user créateur          |
| submitted      | date=aaaa-mm-dd      | NOT NULL, Default=CURDATE()   | Date création recette     |
| tags           | JSON=list[str]       | NOT NULL, Default=[]          | Tags de la recette        |
| nutrition      | JSON=list[5 x float] | NOT NULL, Default=[0,0,0,0,0] | Indice nutritionnelle     |
| n_step         | INTEGER              | NOT NULL, Default=0           | Nombre étape              |
| step           | JSON=list[str]       | NOT NULL, Default=[]          | étape de la recette       |
| description    | TEXT                 | NOT NULL, Default=''          | Description de la recette |
| ingredient     | JSON=list[str]       | NOT NULL, Default=[]          | Liste d'ingrédient        |
| n_ingredient   | INTEGER              | NOT NULL, Default=0           | Nombre ingrédient         |




## ***II) Analyse des données en mode brut sans pré-traitement***

Cette première étape, l'analyse en mode brut se focalise sur le format numérique des données récupérées.
 - Savoir si les dimensions des tableaux sont uniformes.
 - Savoir si les types ne sont pas conformes.

Pour cela, il ne faut pas utiliser à cette étape d'outil comme Pandas qui transforme les données et les convertis dans des types.

### II.0) Chargement des data

In [1]:
import csv
import datetime
import os
import sys
from itertools import islice
from pprint import pprint

from tabulate import tabulate

DATA_FOLDER = '../../data/recipe/'
DATA_FILES = ['RAW_interactions.csv', 'RAW_recipes.csv']
CHARSET = 'UTF-8'


###############################################################################
def normalise_folder(path: str) -> str:
    """Change le chemin en format UNIX (/) et ajoute le / final si inexistant.

    :param str path: Chemin de dossier à normaliser
    :return str: le chemin remis en forme
    """
    path = path.replace('\\', '/')
    if (path[-1] != '/'):
        path = path + '/'
    # end if
    return path
# end def normalise_folder


def take(n, iterable):
    """Return the first n items of the iterable as a list."""
    return list(islice(iterable, n))
#end def take


###############################################################################
# Powershell 
#   $a = Import-Csv -Path .\data\recipe\RAW_interactions.csv -Delimiter ','
#   $b = Import-Csv -Path .\data\recipe\RAW_recipes.csv -Delimiter ','

# Chargement les datas
data = {}
for filename in DATA_FILES:
    data[filename] = []
    file_path = os.path.join(DATA_FOLDER, filename)
    with open(file_path, 'r', encoding=CHARSET) as file:
        spamreader = csv.reader(file, delimiter=',')
        # spamreader = csv.DictReader(file, delimiter=',')
        for row in spamreader:
            data[filename].append(row)

pprint(data[filename][0:2], compact=True)

[['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags', 'nutrition',
  'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients'],
 ['arriba   baked winter squash mexican style', '137739', '55', '47892',
  '2005-09-16',
  "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', "
  "'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', "
  "'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', "
  "'winter', 'dietary', 'christmas', 'seasonal', 'squash']",
  '[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]', '11',
  "['make a choice and proceed with recipe', 'depending on size of squash , "
  "cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle "
  "olive oil or melted butter over each cut squash piece', 'season with "
  "mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , "
  "butter , grated piloncillo over each cut squash piece', 'season with sweet "
  "mexican spice mix', 'bake at 350 degrees

### II.1) Vérification de la consistance de la base de données

In [4]:
# Vérification taille d'une ligne :
count_str_vide = {}
for filename in DATA_FILES:
    header = data[filename][0]
    n_header = len(header)
    print(f'\n** FICHIER {filename}\n')
    print(f'header = {header}')
    print(f'Taille header = {n_header}')
    OK_taille = True
    OK_str = True
    count_str_vide[filename] = [0] * len(header)
    for i, row in enumerate(data[filename]):
        if len(row) != n_header:
            print(f'Taille ligne {i} incorrecte ={len(row)} : {row}')
            OK_taille = False
        for j, col in enumerate(row):
            if col == '':
                if header[j] not in ('review', 'description'):  # champ pouvant être vide souvent
                    print(f'\tChaîne vide à {i} pour {header[j]} ({row[0:2]})')
                count_str_vide[filename][j] += 1
                OK_str = False

    if OK_taille is True:
        print("ANALYSE DQM = Pas de colonne manquante ou en trop")
    else:
        print("ANALYSE DQM = WARNING: Des colonnes sont manquantes ou en trop !")

    if OK_str is True:
        print("ANALYSE DQM = Pas de string vide")
    else:
        print("ANALYSE DQM = Présence de string vide !")
        pprint(count_str_vide[filename])
        for j, col in enumerate(header):
            if count_str_vide[filename][j] != 0:
                print(f'\t{col} = {count_str_vide[filename][j]} vides')



** FICHIER RAW_interactions.csv

header = ['user_id', 'recipe_id', 'date', 'rating', 'review']
Taille header = 5
ANALYSE DQM = Pas de colonne manquante ou en trop
ANALYSE DQM = Présence de string vide
[0, 0, 0, 0, 165]
	review = 165 vides

** FICHIER RAW_recipes.csv

header = ['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients']
Taille header = 12
	Chaîne vide à 722 pour name (['', '368257'])
ANALYSE DQM = Pas de colonne manquante ou en trop
ANALYSE DQM = Présence de string vide
[1, 0, 0, 0, 0, 0, 0, 0, 0, 4979, 0, 0]
	name = 1 vides
	description = 4979 vides


In [22]:
filename = DATA_FILES[1]
header = data[filename][0]
x = data[filename][722]
tmp={header[i]:x[i] for i in range(len(header))}
pprint(tmp, sort_dicts=False)
print(len([i for i in data[filename] if i[3] == x[3] and i[5] == x[5]]))

{'name': '',
 'id': '368257',
 'minutes': '10',
 'contributor_id': '779451',
 'submitted': '2009-04-27',
 'tags': "['15-minutes-or-less', 'time-to-make', 'course', 'preparation', "
         "'low-protein', 'salads', 'easy', 'salad-dressings', 'dietary', "
         "'low-sodium', 'inexpensive', 'low-in-something', '3-steps-or-less']",
 'nutrition': '[1596.2, 249.0, 155.0, 0.0, 2.0, 112.0, 14.0]',
 'n_steps': '6',
 'steps': "['in a bowl , combine ingredients except for olive oil', 'slowly "
          "whisk inches', 'olive oil until thickened', 'great with field "
          "greens', 'makes about 2 / 3', 'cup dressing']",
 'description': '-------------',
 'ingredients': "['lemon', 'honey', 'horseradish mustard', 'garlic clove', "
                "'dried parsley', 'dried basil', 'dried thyme', 'garlic salt', "
                "'black pepper', 'olive oil']",
 'n_ingredients': '10'}
1


**Conclusion :** c'est une recette valide mais elle n'a pas de nom
 - L'user ne la pas re-créer avec un nom cette fois-ci (si oui nous pouvons la retirer sans impact) => Il faut la garder.
 - À étudier, si pour nous nous laissons la string vide ou si nous la transformons en '-' pour éviter des problèmes de manipulation futur. [en tout cas, il faut savoir len(name)=0 / à ne pas diviser ou multiplier !]

### II.2) Vérification homogénisation des types de données

In [34]:
# Vérification des types des colonnes :
header_type = {}
header_type[DATA_FILES[0]] = ['int', 'int', 'date=aaaa-mm-dd', 'int', 'str']
header_type[DATA_FILES[1]] = ['str', 'int', 'int', 'int', 'date=aaaa-mm-dd', 'list=str', 'list=float=7', 'int', 'list=str', 'str', 'list=str', 'int']
count_type_faux = {}
count_car_noprintable = {}
col_min = {}
col_max = {}
tmp_str_col = []
for filename in DATA_FILES:
    header = data[filename][0]
    n_header = len(header)
    print(f'\n** FICHIER {filename}')
    count_type_faux[filename] = [0] * len(header)
    count_car_noprintable[filename] = [0] * len(header)
    col_max[filename] = [None] * len(header)
    col_min[filename] = [None] * len(header)
    for j, col in enumerate(header_type[filename]):
        if 'list=' in header_type[filename][j]:
            h = header_type[filename][j].split('=')
            taille = 1
            if len(h) > 2:
                taille = int(h[2])
            col_min[filename][j] = [None] * taille
            col_max[filename][j] = [None] * taille

    for i, row in enumerate(data[filename]):
        if i == 0:  # Pas de check sur les header
            continue
        for j, col in enumerate(row):
            OK_type = True
            OK_type_split = True
            tmp_str_col = [col]

            if header_type[filename][j] == 'int':  # Vérification des Integer
                if col.isdigit() is False:
                    OK_type = False
                    count_type_faux[filename][j] += 1
                if col_max[filename][j] == None or col_max[filename][j] < int(col):
                    col_max[filename][j] = int(col)
                if col_min[filename][j] == None or col_min[filename][j] > int(col):
                    col_min[filename][j] = int(col)
                continue  # On passe au suivant
            
            if 'date=' in header_type[filename][j]:  # vérification des dates
                if len(col) != len(header_type[filename][j].split('=')[1]):
                    print(f'\t{i} : date mal formée = {col}')  # à voir si parsing OK
                tmp = col.split('-')
                if len(tmp) != 3:  # aaaa et mm et dd
                    OK_type_split = False
                    if len(tmp) < 3:
                        print(f'\t{i} : date mal formée, trop courte = {col}')
                        OK_type = False
                        count_type_faux[filename][j] += 1
                        continue  # On passe au suivant car on ne peut pas aller plus loin
                if not (tmp[0].isdigit() and tmp[1].isdigit() and tmp[2].isdigit()):
                    OK_type = False
                else:
                    try:
                        d = datetime.date(int(tmp[0]), int(tmp[1]), int(tmp[2]))
                        if col_max[filename][j] == None or col_max[filename][j] < d:
                            col_max[filename][j] = d
                        if col_min[filename][j] == None or col_min[filename][j] > d:
                            col_min[filename][j] = d
                    except ValueError:
                        OK_type = False
                if (OK_type is False) or (OK_type_split is False):
                    count_type_faux[filename][j] += 1
                continue  # On passe au suivant

            if 'list=' in header_type[filename][j]:  # vérification des listes ('list=str', 'list=int=5')
                if len(col) < 2 or col[0] != '[' and col[-1] != ']':
                    print(f'\t{i} : liste mal formée = {col}')
                    OK_type = False
                    count_type_faux[filename][j] += 1
                    continue
                l = col[1:-1].split(',')

                h = header_type[filename][j].split('=')
                taille = 1
                if len(h) > 2:
                    taille = int(h[2])
                    if len(l) != taille:
                        print(f'\t{i} : liste de taille incorrecte (!={taille}) = {col}')
                        OK_type = False

                if h[1] == 'str':
                    tmp_str_col = l
                if h[1] == 'float':
                    try:
                        for k, x in enumerate(l):
                            d = float(x)
                            if col_max[filename][j][k] == None or col_max[filename][j][k] < d:
                                col_max[filename][j][k] = d
                            if col_min[filename][j][k] == None or col_min[filename][j][k] > d:
                                col_min[filename][j][k] = d
                    except Exception as e:
                        print(e)
                        OK_type = False
                
                if OK_type is False:
                    count_type_faux[filename][j] += 1
                    continue  # on ne passe pas à la vérification des sous-type de type str

            if 'str' in header_type[filename][j]:  # ('str', list='str') Pas de vérification particulière car déjà str UTF-8 valide
                for k, x in enumerate(tmp_str_col):
                    if x.isprintable() is False:
                        # print(f'{i} : contient des caractères non imprimable !')
                        count_car_noprintable[filename][j] += 1
                    d = len(x)
                    if 'list=' in header_type[filename][j]:
                        if col_max[filename][j][0] == None or col_max[filename][j][0] < d:
                            col_max[filename][j][0] = d
                        if col_min[filename][j][0] == None or col_min[filename][j][0] > d:
                            col_min[filename][j][0] = d
                    else:
                        if col_max[filename][j] == None or col_max[filename][j] < d:
                            col_max[filename][j] = d
                        if col_min[filename][j] == None or col_min[filename][j] > d:
                            col_min[filename][j] = d


    print(header_type[filename])
    if OK_type is True:
        print("ANALYSE DQM = Pas de problème de type avec ceux décidés")
    else:
        print("ANALYSE DQM = WARNING: Des colonnes ont des types non conformes !")
        pprint(count_type_faux[filename])
        for j, col in enumerate(header):
            if count_type_faux[filename][j] != 0:
                print(f'\t{col}({header_type[filename][j]}) = {count_type_faux[filename][j]} type invalide')
    print('\nValeur min / max des nombres ou len(str) :')
    for j, col in enumerate(header):
        if col_min[filename][j] is not None :
            print(f'\t{col}({header_type[filename][j]}) = {col_min[filename][j]} / {col_max[filename][j]}')


** FICHIER RAW_interactions.csv
['int', 'int', 'date=aaaa-mm-dd', 'int', 'str']
ANALYSE DQM = Pas de problème de type avec ceux décidés
Valeur min / max des nombres ou len(str) :
	user_id(int) = 1533 / 2002372706
	recipe_id(int) = 38 / 537716
	date(date=aaaa-mm-dd) = 2000-01-25 / 2018-12-20
	rating(int) = 0 / 5
	review(str) = 0 / 6972
** FICHIER RAW_recipes.csv
['str', 'int', 'int', 'int', 'date=aaaa-mm-dd', 'list=str', 'list=float=7', 'int', 'list=str', 'str', 'list=str', 'int']
ANALYSE DQM = Pas de problème de type avec ceux décidés
Valeur min / max des nombres ou len(str) :
	name(str) = 0 / 85
	id(int) = 38 / 537716
	minutes(int) = 0 / 2147483647
	contributor_id(int) = 27 / 2002289981
	submitted(date=aaaa-mm-dd) = 1999-08-06 / 2018-12-04
	tags(list=str) = [2] / [71]
	nutrition(list=float=7) = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] / [434360.2, 17183.0, 362729.0, 29338.0, 6552.0, 10395.0, 36098.0]
	n_steps(int) = 0 / 145
	steps(list=str) = [0] / [625]
	description(str) = 0 / 6267
	ingre

#### Remarques :

- Les clés primaires et étrangères sont bien des entiers positifs. La taille reste en int32 mais il faudrait mieux prendre soit uint32 soit int64 pour les manipuler.
- La valeur max des minutes == INT32_MAX_VALUE. D'une part ce champ est représenté dans le système par un int32 et d'autre part la valeur mise par l'utilisateur était le max ou trop grande.
- Les autres élements ne présente pas problème apparent pour une conversion de type automatique.

### II.3) Vérification unicité des clés et existance (primaires, étrangère, NOT NULL)


Avec les analyses précentes, nous avons déjà vérifié leur existance, il suffit maintenant de vérifier leur unicité.

In [2]:
count_unique_faux = {}
BDD_OK = True
keys_unique = True
col_key = {}
col_key[DATA_FILES[0]] = [('user_id', 'recipe_id', 'date')]
col_key[DATA_FILES[1]] = ['id']

for filename in DATA_FILES:
    BDD_OK = True
    keys_unique = True
    print(f'\n** FICHIER {filename}')
    header = data[filename][0]
    count_unique_faux[filename] = {}
    if filename == DATA_FILES[1]:
        for col in col_key[filename]:
            count_unique_faux[filename][col] = 0
            idx = header.index(col)
            n = len(data[filename][idx])
            n_set = len(set(data[filename][idx]))
            if n != n_set:
                print(f'\t{col} : présence de doublons ({n - n_set})')
                count_unique_faux[filename][col] += 1
                keys_unique = False
                BDD_OK = False
    if filename == DATA_FILES[0]:   # 'RAW_interactions.csv':
        keys = [''.join([''.join(str(x)) for x in i]) for i in [x[0:3] for x in data[filename]]]
        n = len(keys)
        n_set = len(set(keys))
        if n != n_set:
            print(f'\tuser_id,recipe_id,date : présence de doublons ({n - n_set})')
            keys = [''.join([''.join(str(x)) for x in i]) for i in [x[0:4] for x in data[filename]]]
            n = len(keys)
            n_set = len(set(keys))
            if n != n_set:
                print(f'\t\tuser_id,recipe_id,date,rating : présence de doublons ({n - n_set})')
                keys = [''.join([''.join(str(x)) for x in i]) for i in data[filename]]
                n = len(keys)
                n_set = len(set(keys))
                if n != n_set:
                    print(f'\t=>table complète : ERREUR présence de doublons ({n - n_set})')
                    BDD_OK = False
            count_unique_faux[filename][col] += 1
            keys_unique = False
    
    if BDD_OK is False:
        print("ANALYSE DQM = ERREUR: Base de données a une structure invalide concernant les contraites de clés !!")
    if keys_unique is True:
        print("ANALYSE DQM = Pas de problème de contrainte BDD")
    else:
        print("ANALYSE DQM = WARNING: Des clés primaires supposées contiennent des doublons, donc hypothèse fausse du choix des clés primaires")
        pprint(count_unique_faux[filename])
        for j, col in enumerate(header):
            if count_type_faux[filename][j] != 0:
                print(f'\t({col}) = {count_unique_faux[filename][col]} non unique')



** FICHIER RAW_interactions.csv
ANALYSE DQM = Pas de problème de contrainte BDD

** FICHIER RAW_recipes.csv
ANALYSE DQM = Pas de problème de contrainte BDD


### II.4) Vérification existances clés étrangères


Ici nous pouvons regarder seulement la relation REVIEW->RECIPE avec les données possédées.

Permet de déterminer les erreurs de suppression non en cascade.

In [11]:
count_missing_recipe = 0
filename = DATA_FILES[1]  # 'RAW_recipes.csv'
list_recipe_id = [int(row[1]) for row in data[filename][1:]]
filename = DATA_FILES[0]  # 'RAW_interactions.csv'
list_review_recipe_id = [int(row[1]) for row in data[filename][1:]]
print("Rappel :")
print("\tNb recette (unique par def.) :", len(set(list_recipe_id)))
print("\tNb review : ", len(list_review_recipe_id))
print("\tNb recette unique dans les review :", len(set(list_review_recipe_id)))
taille = len(set.union(set(list_recipe_id), set(list_review_recipe_id)))
if taille == len(list_recipe_id):
    print("ANALYSE DQM = Pas de référence à une recette inexistante.")
else:
    print(f"ANALYSE DQM = ERREUR: référence à recette (nb inconnu = {taille - len(list_recipe_id)})")


Rappel :
	Nb recette (unique par def.) : 231637
	Nb review :  1132367
	Nb recette unique dans les review : 231637
ANALYSE DQM = Pas de référence à une recette inexistante.


### II.5) Conclusion de l'analyse brute

Globalement, modélisation des données cohérente et données propres sans valeurs manquantes techniques.

- Confirmation de notre analyse conceptuelle (§I) concernant les objets, leurs relations et leurs représentations techniques ;
  - De plus, les types techniques des données sont conformes à notre modélisation 
- Pas de problème concernant les données ou la structure, ni d'incohérence technique
- Pas de données corrompues ou manquante, seulement constant d'avertissement :
  - Il y a 1 recette sans nom (unique) mais réelle.
  - Il y a le champ minutes dont la valeur MAX d'encodage est atteint (INT32)

**Remarques :**
 - champ `name` : pour les analyses futures en prendre en compte que len(name)=0, sinon la renommer : '-' ;
 - champ `minute` : des valeurs au max (INT32_MAX_VALUE) soit voulu par l'user ou trop grande


#### Résumé des résultats :

![Figure 1 : Modélisation de la base](Data_UML.png)

- **FICHIER RAW_interactions.csv**

|         | user_id    | recipe_id   | date            | rating   | review   |
|:--------|:-----------|:------------|:----------------|:---------|:---------|
| *Type*    | *int64*      | *int*         | *date=aaaa-mm-dd* | *int*      | *str*      |
| Nb vide | 0          | 0           | 0               | 0        | 165      |
| Val min | 1533       | 38          | 2000-01-25      | 0        | 0        |
| Val max | 2002372706 | 537716      | 2018-12-20      | 5        | 6972     |

- **FICHIER RAW_recipes.csv**

|         | name   | id     | minutes    | contributor_id   | submitted       | tags     | nutrition                                                        | n_steps   | steps    | description   | ingredients   | n_ingredients   |
|:--------|:-------|:-------|:-----------|:-----------------|:----------------|:---------|:-----------------------------------------------------------------|:----------|:---------|:--------------|:--------------|:----------------|
| *Type*    | *str*    | *int*    | *int*        | *int64*            | *date=aaaa-mm-dd* | *list=str* | *list=float=7*                                                     | *int*       | *list=str* | *str*           | *list=str*      | *int*             |
| Nb vide | 1      | 0      | 0          | 0                | 0               | 0        | 0                                                                | 0         | 0        | 4979          | 0             | 0               |
| Val min | 0      | 38     | 0          | 27               | 1999-08-06      | [2]      | [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                              | 0         | [0]      | 0             | [5]           | 1               |
| Val max | 85     | 537716 | 2147483647 | 2002289981       | 2018-12-04      | [71]     | [434360.2, 17183.0, 362729.0, 29338.0, 6552.0, 10395.0, 36098.0] | 145       | [625]    | 6267          | [76]          | 43              |


In [15]:
resultat_str = {}
index = ['Type', 'Nb vide', 'Val min', 'Val max']
filename = DATA_FILES[0]  #'RAW_interactions.csv'
header = ['user_id', 'recipe_id', 'date', 'rating', 'review']
nb_vide = [0, 0, 0, 0, 165]
data_type = ['int64', 'int', 'date=aaaa-mm-dd', 'int', 'str']
data_min = [1533, 38, '2000-01-25', 0, 0]
data_max = [2002372706, 537716, '2018-12-20', 5, 6972]
resultat_str[filename] = tabulate([data_type, nb_vide, data_min, data_max], showindex=index, headers=header, tablefmt='pipe')

filename = DATA_FILES[1]  # 'RAW_recipes.csv'
header = ['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients']
nb_vide = [1, 0, 0, 0, 0, 0, 0, 0, 0, 4979, 0, 0]
data_type = ['str', 'int', 'int', 'int64', 'date=aaaa-mm-dd', 'list=str', 'list=float=7', 'int', 'list=str', 'str', 'list=str', 'int']
data_min = [0, 38, 0, 27, '1999-08-06', [2], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0, [0], 0, [5], 1]
data_max = [85, 537716, 2147483647, 2002289981, '2018-12-04', [71], [434360.2, 17183.0, 362729.0, 29338.0, 6552.0, 10395.0, 36098.0], 145, [625], 6267, [76], 43]
resultat_str[filename] = tabulate([data_type, nb_vide, data_min, data_max], showindex=index, headers=header, tablefmt='pipe')

for filename in DATA_FILES:
    print(f'\n** FICHIER {filename}')
    print(resultat_str[filename])
    print()



** FICHIER RAW_interactions.csv
|         | user_id    | recipe_id   | date            | rating   | review   |
|:--------|:-----------|:------------|:----------------|:---------|:---------|
| Type    | int64      | int         | date=aaaa-mm-dd | int      | str      |
| Nb vide | 0          | 0           | 0               | 0        | 165      |
| Val min | 1533       | 38          | 2000-01-25      | 0        | 0        |
| Val max | 2002372706 | 537716      | 2018-12-20      | 5        | 6972     |


** FICHIER RAW_recipes.csv
|         | name   | id     | minutes    | contributor_id   | submitted       | tags     | nutrition                                                        | n_steps   | steps    | description   | ingredients   | n_ingredients   |
|:--------|:-------|:-------|:-----------|:-----------------|:----------------|:---------|:-----------------------------------------------------------------|:----------|:---------|:--------------|:--------------|:----------------|
| 

## III Analyse de la cohérence conceptuelle

Notamment vérifier les lien d'antériorité des objects (au sein d'un même object id/date et entre les objects (recette/review)) et d'existante des clés étrangères.

In [13]:
# Préparation :
count_recette_ante = {}
tmp = {}  # dict[clé_primaire: date] ou dict[self.date: foreign.date]
id_recette_updated = {}


filename = DATA_FILES[0]  #'RAW_interactions.csv'
list_review_recipe_id = [int(row[1]) for row in data[filename][1:]]
set_recette_in_review = set(list_review_recipe_id)
lien_review = [[datetime.date.fromisoformat(x[2]), int(x[1]), i] for i, x in enumerate(data[filename][1:])]
lien_review = sorted(lien_review)

filename = DATA_FILES[1]  #'RAW_recipes.csv'
lien_recette = {int(i[1]):datetime.date.fromisoformat(i[4]) for i in data[filename][1:]}
fusion_rec_rev = {int(row[1]):{'date':datetime.date.fromisoformat(row[4]), 'reviews':[]} for row in data[filename][1:] if int(row[1]) in set_recette_in_review}

for lien in lien_review:
    id_recette = lien[1]
    id_review = lien[2]
    date_review = lien[0]
    fusion_rec_rev[id_recette]['reviews'].append([date_review, id_review])


In [14]:
# Cohérence des recettes :
# key -> date, seulement valable pour recipe car pour review la date est une clé primaire

filename = DATA_FILES[1]  # 'RAW_recipes.csv'
count_recette_ante[filename] = 0
print(f'\n** FICHIER {filename}')
tmp_id = None
tmp_date = None
for id_recette in sorted(lien_recette):
    date_recette = lien_recette[id_recette]
    if tmp_id == None:  # Initialisation
        tmp_date = date_recette
        tmp_id = id_recette
    if tmp_id > id_recette:
        print(f"{id_recette} : Erreur d'ordre du dictionnaire")
        break
    tmp_id = id_recette
    if tmp_date > date_recette:
        print(f"{id_recette} ({date_recette} < i-1={tmp_date}) : Recette antérieure")
        count_recette_ante[filename] += 1
        id_recette_updated[id_recette] = {'date':date_recette, 'next_date':tmp_date}



** FICHIER RAW_recipes.csv
45 (1999-08-06 < i-1=1999-08-09) : Recette antérieure
108 (1999-08-06 < i-1=1999-08-09) : Recette antérieure
133 (1999-08-06 < i-1=1999-08-09) : Recette antérieure
190 (1999-08-06 < i-1=1999-08-09) : Recette antérieure
199 (1999-08-07 < i-1=1999-08-09) : Recette antérieure
203 (1999-08-06 < i-1=1999-08-09) : Recette antérieure
213 (1999-08-06 < i-1=1999-08-09) : Recette antérieure
226 (1999-08-08 < i-1=1999-08-09) : Recette antérieure
543 (1999-08-07 < i-1=1999-08-09) : Recette antérieure
563 (1999-08-06 < i-1=1999-08-09) : Recette antérieure
565 (1999-08-06 < i-1=1999-08-09) : Recette antérieure
571 (1999-08-08 < i-1=1999-08-09) : Recette antérieure
583 (1999-08-06 < i-1=1999-08-09) : Recette antérieure
589 (1999-08-07 < i-1=1999-08-09) : Recette antérieure
590 (1999-08-07 < i-1=1999-08-09) : Recette antérieure
630 (1999-08-07 < i-1=1999-08-09) : Recette antérieure
685 (1999-08-07 < i-1=1999-08-09) : Recette antérieure
707 (1999-08-08 < i-1=1999-08-09) : Re

In [15]:

filename = DATA_FILES[0]  # 'RAW_interactions.csv'
id_review_ante = {}
count_recette_ante[filename] = 0
print(f'\n** FICHIER {filename}')
for id_recette in fusion_rec_rev:
    date_recette = fusion_rec_rev[id_recette]['date']
    for review in fusion_rec_rev[id_recette]['reviews']:
        date_review =  review[0]
        id_review =  review[1]
        if date_review < date_recette:
            print(f"{id_recette} : ({date_review} < recette={date_recette}) : Recette avec review antérieur (id={id_review})")
            count_recette_ante[filename] += 1
            if id_recette not in id_recette_updated:
                id_review_ante[id_review] = {'date_review':date_review, 'date_recette':date_recette, 'id_recette':id_recette}



** FICHIER RAW_interactions.csv
161370 (2006-02-22 < recette=2006-03-24) : Review antérieur (id=115787)
161370 (2006-02-25 < recette=2006-03-24) : Review antérieur (id=115788)
161370 (2006-02-26 < recette=2006-03-24) : Review antérieur (id=115789)
161370 (2006-02-27 < recette=2006-03-24) : Review antérieur (id=115790)
161370 (2006-03-02 < recette=2006-03-24) : Review antérieur (id=115791)
161370 (2006-03-06 < recette=2006-03-24) : Review antérieur (id=115792)
161370 (2006-03-07 < recette=2006-03-24) : Review antérieur (id=115793)
161370 (2006-03-11 < recette=2006-03-24) : Review antérieur (id=115794)
161370 (2006-03-12 < recette=2006-03-24) : Review antérieur (id=115795)
161370 (2006-03-13 < recette=2006-03-24) : Review antérieur (id=115796)
161370 (2006-03-19 < recette=2006-03-24) : Review antérieur (id=115797)
161370 (2006-03-22 < recette=2006-03-24) : Review antérieur (id=115798)
161370 (2006-03-23 < recette=2006-03-24) : Review antérieur (id=115799)
32204 (2002-06-24 < recette=200

In [53]:
# a : => sorted([[id_review_ante[x]['date_review'].isoformat(), id_review_ante[x]['date_recette'].isoformat(), id_review_ante[x]['id_recette'], x] for x in id_review_ante])
nb_recette_impact = len(set([id_review_ante[x]['id_recette'] for x in id_review_ante]))

print("** CONCLUSION :")
print(f"Nombre recette antérieure techniquement : {count_recette_ante[DATA_FILES[1]]}")
tmp = [id_recette_updated[x]['date'] for x in id_recette_updated]
tmp_min = min(tmp)
tmp_max = max(tmp)
print(f"\tDate_min={tmp_min.isoformat()} / Date_max={tmp_max.isoformat()}")
print(f"Nombre recette ayant des reviews antérieurs : {nb_recette_impact}")
print(f"  -> Nombre de review antérieur associé : {count_recette_ante[DATA_FILES[0]]}")
tmp = [id_review_ante[x]['date_recette'] for x in id_review_ante]
tmp_min = min(tmp)
tmp_max = max(tmp)
print(f"\tRecette : Date_min={tmp_min.isoformat()} / Date_max={tmp_max.isoformat()}")
tmp = [id_review_ante[x]['date_review'] for x in id_review_ante]
tmp_min = min(tmp)
tmp_max = max(tmp)
print(f"\tReview : Date_min={tmp_min.isoformat()} / Date_max={tmp_max.isoformat()}\n")
print(tabulate([data[DATA_FILES[0]][x] for i, x in enumerate(id_review_ante) if i < 5], headers=data[DATA_FILES[0]][0]))


** CONCLUSION :
Nombre recette antérieure techniquement : 31
	Date_min=1999-08-06 / Date_max=1999-08-08
Nombre recette ayant des reviews antérieurs : 380
  -> Nombre de review antérieur associé : 1246
	Recette : Date_min=2001-09-26 / Date_max=2011-11-21
	Review : Date_min=2000-05-21 / Date_max=2011-08-08

  user_id    recipe_id  date          rating  review
---------  -----------  ----------  --------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   137050       145595  2007-11-12         5  This is truly an elegant and beautiful dessert.  I made this for my best friend's birthday dinn

**Conclusion :**
- *Controle technique sur les clés primaires (seul RAW_recipes concerné)*
  - 31 recettes ont leur identifiant unique (de création dans la BDD) antérieur à leur date.
  - Cela veut dire que ces enregistrements dans la BDD ont été mis à jour, car des sauts d'id existe dans la BDD et ce champ est en auto-incrément.
  - **=> Aucune conséquence**, car les commentaires commencent dès 2000, et la totatlité de ces recettes sont de 1999. Et aussi, car c'est un élément interne non pertinant pour une analyse.
- *Contrôle sur les clés étrangères et donc le lien RECETTE->REVIEW*
  - 1246 commentaires, sur 380 recettes, avec une date postérieure à la date de sa recette.
  - Concerne une période entre 2000-2011 sur la période d'étude de 1999-2018.
  - Les commentaires sont pertinants, il ne faudrait pas les retirer.
  - **=> Pas de données manquantes mais vigilance pour l'analyse**
    - Il faut garder ces commentaires lorsqu'on travaille dessus.
    - Par contre, si on étudie la temporalité de ces commentaires, il faut effectuer une normalisation de la date de ceux-ci par rapport à la date de la recette (ex. les remplacer par la date de la recette si antérieur).